In [1]:
import time

In [2]:
start_time = time.time()
import math
import requests
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, InputLayer, Dropout, Conv1D, Flatten, Reshape, MaxPooling1D, BatchNormalization,
    Conv2D, GlobalMaxPooling2D, Lambda, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from pathlib import Path
load_time = time.time() - start_time
print(f"\nlibraries import completed in{load_time}\n")

2025-01-17 13:40:21.957375: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 13:40:25.048806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737150026.708037   94864 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737150027.045665   94864 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 13:40:29.495190: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr


libraries import completed in142.23748993873596



In [20]:
# Constants and paths
WEIGHTS_PATH = './transfer-learning-weights/edgeimpulse/MobileNetV2.0_35.96x96.grayscale.bsize_64.lr_0_005.epoch_260.val_loss_3.10.val_accuracy_0.35.hdf5'
ROOT_URL = 'https://cdn.edgeimpulse.com/'
INPUT_SHAPE = (96, 96, 1)
BATCH_SIZE = 32
EPOCHS = 40
LEARNING_RATE = 0.0005
FINE_TUNE_EPOCHS = 15
FINE_TUNE_PERCENTAGE = 65
TRAIN_DIR = './CANS-DATA/TRAIN'  # Update this with your train dataset directory
VALIDATION_DIR = './CANS-DATA/TEST'  # Update this with your validation dataset directory

# Download pretrained weights if not available
p = Path(WEIGHTS_PATH)
if not p.exists():
    print(f"Pretrained weights {WEIGHTS_PATH} unavailable; downloading...")
    if not p.parent.exists():
        p.parent.mkdir(parents=True)
    weights_data = requests.get(ROOT_URL + WEIGHTS_PATH[2:]).content
    with open(WEIGHTS_PATH, 'wb') as f:
        f.write(weights_data)
    print(f"Pretrained weights {WEIGHTS_PATH} downloaded successfully.")

# Load dataset
# train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     TRAIN_DIR,
#     image_size=INPUT_SHAPE[:2],
#     batch_size=BATCH_SIZE,
#     label_mode='categorical'
# )

# validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     VALIDATION_DIR,
#     image_size=INPUT_SHAPE[:2],
#     batch_size=BATCH_SIZE,
#     label_mode='categorical'
# )
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(96, 96),  # Resize images to 96x96
    batch_size=BATCH_SIZE,
    color_mode='grayscale',  # Ensure images are grayscale
    label_mode='categorical'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    VALIDATION_DIR,
    image_size=(96, 96),  # Resize validation images to 96x96
    batch_size=BATCH_SIZE,
    color_mode='grayscale',  # Ensure validation images are grayscale
    label_mode='categorical'
)


Found 361 files belonging to 3 classes.
Found 92 files belonging to 3 classes.


In [11]:
# Load the base MobileNetV2 model with pretrained weights- aplha 0.35 works good but may prefer lower later
base_model = tf.keras.applications.MobileNetV2(
    input_shape=INPUT_SHAPE, alpha=0.35, weights=WEIGHTS_PATH
)
print("base model loaded")
base_model.trainable = True  # True works good

# Build the final model
model = Sequential()
model.add(InputLayer(input_shape=INPUT_SHAPE, name='x_input'))
last_layer_index = -3
model.add(Model(inputs=base_model.inputs, outputs=base_model.layers[last_layer_index].output))
model.add(Reshape((-1, model.layers[-1].output.shape[3])))
model.add(Dense(8, activation='relu'))  # 8 works good
model.add(Dropout(0.2)) # 0.2 works good
model.add(Flatten())
model.add(Dense(3, activation='softmax'))  # 3 classes (vertical, horizontal, nothing)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

# Callbacks (optional)
callbacks = [
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    #EarlyStopping(patience=3, restore_best_weights=True)
]

# Train the model
print("Training the model...")
model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS, verbose=2, callbacks=callbacks)

# Fine-tuning the model
print(f'Fine-tuning the model for {FINE_TUNE_EPOCHS} epochs...')
model = tf.keras.models.load_model('best_model.h5')

# Unfreeze the base model layers and freeze layers before the fine-tune layer
model_layer_count = len(model.layers)
fine_tune_from = math.ceil(model_layer_count * ((100 - FINE_TUNE_PERCENTAGE) / 100))

model.trainable = True
for layer in model.layers[:fine_tune_from]:
    layer.trainable = False

# Recompile the model after unfreezing
model.compile(optimizer=Adam(learning_rate=0.000045),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(train_dataset, epochs=FINE_TUNE_EPOCHS, verbose=2, validation_data=validation_dataset, callbacks=callbacks)

print("Model training and fine-tuning completed successfully.")

base model loaded
Training the model...
Epoch 1/80


/mnt/c/Users/rtill/AI-venv/AIvenv/lib/python3.12/site-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_2010']
Received: inputs=Tensor(shape=(None, 96, 96, 1))
  warnings.warn(msg)


12/12 - 41s - 3s/step - accuracy: 0.5845 - loss: 0.9651 - val_accuracy: 0.3913 - val_loss: 1.0559
Epoch 2/80


12/12 - 1s - 101ms/step - accuracy: 0.8892 - loss: 0.2957 - val_accuracy: 0.4674 - val_loss: 1.0433
Epoch 3/80
12/12 - 0s - 38ms/step - accuracy: 0.9391 - loss: 0.1642 - val_accuracy: 0.4239 - val_loss: 1.5490
Epoch 4/80
12/12 - 0s - 32ms/step - accuracy: 0.9640 - loss: 0.0857 - val_accuracy: 0.4348 - val_loss: 1.9611
Epoch 5/80


12/12 - 1s - 95ms/step - accuracy: 0.9917 - loss: 0.0435 - val_accuracy: 0.5435 - val_loss: 1.4638
Epoch 6/80


...

12/12 - 1s - 104ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.9130 - val_loss: 0.3279
Epoch 25/80


12/12 - 1s - 119ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.9239 - val_loss: 0.3260
Epoch 26/80
12/12 - 0s - 36ms/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.9130 - val_loss: 0.3752
Epoch 27/80
12/12 - 0s - 39ms/step - accuracy: 1.0000 - loss: 0.0029 - val_accuracy: 0.9130 - val_loss: 0.3836
Epoch 28/80
12/12 - 0s - 41ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.9239 - val_loss: 0.3391
Epoch 29/80


12/12 - 1s - 95ms/step - accuracy: 0.9972 - loss: 0.0090 - val_accuracy: 0.9348 - val_loss: 0.2684
Epoch 30/80
12/12 - 1s - 45ms/step - accuracy: 0.9945 - loss: 0.0141 - val_accuracy: 0.9022 - val_loss: 0.2697
Epoch 31/80
12/12 - 0s - 33ms/step - accuracy: 0.9917 - loss: 0.0244 - val_accuracy: 0.9130 - val_loss: 0.2424
Epoch 32/80
12/12 - 1s - 44ms/step - accuracy: 0.9945 - loss: 0.0206 - val_accuracy: 0.9239 - val_loss: 0.2583
Epoch 33/80
12/12 - 0s - 40ms/step - accuracy: 0.9972 - loss: 0.0163 - val_accuracy: 0.8804 - val_loss: 0.3651
Epoch 34/80
12/12 - 1s - 44ms/step - accuracy: 0.9972 - loss: 0.0094 - val_accuracy: 0.8696 - val_loss: 0.4346
Epoch 35/80
12/12 - 0s - 35ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.8913 - val_loss: 0.4243
Epoch 36/80
12/12 - 1s - 46ms/step - accuracy: 0.9972 - loss: 0.0054 - val_accuracy: 0.9130 - val_loss: 0.4145
Epoch 37/80
12/12 - 0s - 32ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 0.9130 - val_loss: 0.4295
Epoch 38/80
1

12/12 - 1s - 116ms/step - accuracy: 0.9917 - loss: 0.0398 - val_accuracy: 0.9457 - val_loss: 0.2050
Epoch 40/80
12/12 - 2s - 141ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.8261 - val_loss: 0.7498
Epoch 41/80
12/12 - 0s - 35ms/step - accuracy: 0.9889 - loss: 0.0242 - val_accuracy: 0.8804 - val_loss: 0.3941
Epoch 42/80
12/12 - 0s - 32ms/step - accuracy: 1.0000 - loss: 0.0052 - val_accuracy: 0.9022 - val_loss: 0.3716
Epoch 43/80
12/12 - 0s - 42ms/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 0.8370 - val_loss: 0.7490
Epoch 44/80
12/12 - 0s - 31ms/step - accuracy: 0.9972 - loss: 0.0112 - val_accuracy: 0.8804 - val_loss: 0.5855
Epoch 45/80
12/12 - 0s - 35ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9239 - val_loss: 0.4738
Epoch 46/80
12/12 - 0s - 32ms/step - accuracy: 0.9972 - loss: 0.0079 - val_accuracy: 0.9239 - val_loss: 0.5017
Epoch 47/80
12/12 - 0s - 37ms/step - accuracy: 1.0000 - loss: 0.0028 - val_accuracy: 0.9130 - val_loss: 0.4814
Epoch 48/80

Epoch 1/15
12/12 - 8s - 689ms/step - accuracy: 0.9889 - loss: 0.0295 - val_accuracy: 0.9457 - val_loss: 0.2046
Epoch 2/15
12/12 - 1s - 47ms/step - accuracy: 0.9972 - loss: 0.0181 - val_accuracy: 0.9457 - val_loss: 0.2040
Epoch 3/15
12/12 - 1s - 45ms/step - accuracy: 0.9834 - loss: 0.0257 - val_accuracy: 0.9457 - val_loss: 0.2034
Epoch 4/15
12/12 - 0s - 40ms/step - accuracy: 0.9972 - loss: 0.0128 - val_accuracy: 0.9457 - val_loss: 0.2029
Epoch 5/15
12/12 - 0s - 34ms/step - accuracy: 0.9972 - loss: 0.0270 - val_accuracy: 0.9457 - val_loss: 0.2022
Epoch 6/15
12/12 - 0s - 34ms/step - accuracy: 0.9889 - loss: 0.0431 - val_accuracy: 0.9457 - val_loss: 0.2016
Epoch 7/15
12/12 - 0s - 35ms/step - accuracy: 0.9917 - loss: 0.0144 - val_accuracy: 0.9457 - val_loss: 0.2009
Epoch 8/15
12/12 - 0s - 34ms/step - accuracy: 0.9889 - loss: 0.0258 - val_accuracy: 0.9457 - val_loss: 0.2010
Epoch 9/15
12/12 - 0s - 30ms/step - accuracy: 0.9889 - loss: 0.0264 - val_accuracy: 0.9457 - val_loss: 0.2002
Epoch 10/

In [26]:
## My beautiful custom model
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

print("building custom base model...")
base_model = Sequential()
base_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=INPUT_SHAPE))
base_model.add(MaxPooling2D((2, 2)))
base_model.add(Conv2D(64, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2, 2)))
base_model.add(Conv2D(128, (3, 3), activation='relu'))
base_model.add(MaxPooling2D((2, 2)))

# Add the custom layers for the final model
model = Sequential()
model.add(InputLayer(input_shape=INPUT_SHAPE, name='x_input'))
model.add(base_model)  # Use the custom CNN base model
model.add(Flatten())  # Flatten the output of the base model
model.add(Dense(32, activation='relu'))  # Dense layer with 128 neurons
model.add(Dropout(0.2))  # Dropout layer to avoid overfitting
model.add(Dense(3, activation='softmax'))  # Output layer with 3 classes (vertical, horizontal, nothing)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

# Callbacks (optional)
callbacks = [
    ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max'),
    # EarlyStopping(patience=3, restore_best_weights=True)
]

# Train the model
print("Training the model...")
model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS, verbose=2, callbacks=callbacks)

# Fine-tuning the model
print(f'Fine-tuning the model for {FINE_TUNE_EPOCHS} epochs...')
model = tf.keras.models.load_model('best_model.h5')

# Unfreeze the custom layers for fine-tuning
model.trainable = True
for layer in model.layers[:len(model.layers)-2]:
    layer.trainable = False

# Recompile the model after unfreezing
model.compile(optimizer=Adam(learning_rate=0.000045),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(train_dataset, epochs=FINE_TUNE_EPOCHS, verbose=2, validation_data=validation_dataset, callbacks=callbacks)

print("Model training and fine-tuning completed successfully.")


building custom base model...
Training the model...
Epoch 1/40


12/12 - 4s - 336ms/step - accuracy: 0.3767 - loss: 9.5524 - val_accuracy: 0.5000 - val_loss: 1.2598
Epoch 2/40
12/12 - 1s - 56ms/step - accuracy: 0.4875 - loss: 1.1455 - val_accuracy: 0.4239 - val_loss: 1.0024
Epoch 3/40


12/12 - 1s - 45ms/step - accuracy: 0.5679 - loss: 0.9305 - val_accuracy: 0.6630 - val_loss: 0.8676
Epoch 4/40


12/12 - 0s - 19ms/step - accuracy: 0.6150 - loss: 0.8348 - val_accuracy: 0.6739 - val_loss: 0.8381
Epoch 5/40
12/12 - 0s - 40ms/step - accuracy: 0.7119 - loss: 0.6858 - val_accuracy: 0.6304 - val_loss: 0.8335
Epoch 6/40
12/12 - 0s - 41ms/step - accuracy: 0.7479 - loss: 0.6132 - val_accuracy: 0.6630 - val_loss: 0.7620
Epoch 7/40
12/12 - 1s - 46ms/step - accuracy: 0.7673 - loss: 0.5411 - val_accuracy: 0.6739 - val_loss: 0.7803
Epoch 8/40
12/12 - 0s - 38ms/step - accuracy: 0.8144 - loss: 0.4451 - val_accuracy: 0.6739 - val_loss: 0.7781
Epoch 9/40


12/12 - 1s - 47ms/step - accuracy: 0.8504 - loss: 0.3762 - val_accuracy: 0.7283 - val_loss: 0.7447
Epoch 10/40
12/12 - 1s - 42ms/step - accuracy: 0.8726 - loss: 0.3284 - val_accuracy: 0.6739 - val_loss: 0.8114
Epoch 11/40
12/12 - 1s - 42ms/step - accuracy: 0.8947 - loss: 0.2647 - val_accuracy: 0.6739 - val_loss: 0.8734
Epoch 12/40
12/12 - 0s - 35ms/step - accuracy: 0.9252 - loss: 0.2112 - val_accuracy: 0.6957 - val_loss: 0.7211
Epoch 13/40
12/12 - 0s - 41ms/step - accuracy: 0.8809 - loss: 0.2344 - val_accuracy: 0.6304 - val_loss: 0.8742
Epoch 14/40
12/12 - 1s - 46ms/step - accuracy: 0.8975 - loss: 0.2173 - val_accuracy: 0.6304 - val_loss: 0.9440
Epoch 15/40
12/12 - 0s - 6ms/step - accuracy: 0.9197 - loss: 0.1858 - val_accuracy: 0.6739 - val_loss: 0.8362
Epoch 16/40
12/12 - 0s - 40ms/step - accuracy: 0.8864 - loss: 0.2035 - val_accuracy: 0.7065 - val_loss: 0.7204
Epoch 17/40
12/12 - 1s - 44ms/step - accuracy: 0.9280 - loss: 0.1612 - val_accuracy: 0.7065 - val_loss: 0.6913
Epoch 18/40
12

12/12 - 1s - 44ms/step - accuracy: 0.9252 - loss: 0.1414 - val_accuracy: 0.7500 - val_loss: 0.8116
Epoch 22/40
12/12 - 0s - 40ms/step - accuracy: 0.9363 - loss: 0.1648 - val_accuracy: 0.7174 - val_loss: 0.8432
Epoch 23/40
12/12 - 1s - 44ms/step - accuracy: 0.9086 - loss: 0.1691 - val_accuracy: 0.7283 - val_loss: 0.7898
Epoch 24/40
12/12 - 0s - 34ms/step - accuracy: 0.9612 - loss: 0.1017 - val_accuracy: 0.7391 - val_loss: 0.7829
Epoch 25/40
12/12 - 1s - 44ms/step - accuracy: 0.9557 - loss: 0.1379 - val_accuracy: 0.7391 - val_loss: 0.7644
Epoch 26/40
12/12 - 0s - 5ms/step - accuracy: 0.9391 - loss: 0.1406 - val_accuracy: 0.6957 - val_loss: 0.7612
Epoch 27/40
12/12 - 0s - 32ms/step - accuracy: 0.9363 - loss: 0.1508 - val_accuracy: 0.7391 - val_loss: 0.8911
Epoch 28/40
12/12 - 1s - 44ms/step - accuracy: 0.9612 - loss: 0.1168 - val_accuracy: 0.6739 - val_loss: 1.0610
Epoch 29/40
12/12 - 1s - 45ms/step - accuracy: 0.9612 - loss: 0.1366 - val_accuracy: 0.7391 - val_loss: 0.8284
Epoch 30/40


12/12 - 1s - 53ms/step - accuracy: 0.9474 - loss: 0.1441 - val_accuracy: 0.7609 - val_loss: 0.9154
Epoch 31/40
12/12 - 1s - 50ms/step - accuracy: 0.9557 - loss: 0.1258 - val_accuracy: 0.7609 - val_loss: 0.9694
Epoch 32/40
12/12 - 1s - 43ms/step - accuracy: 0.9391 - loss: 0.1208 - val_accuracy: 0.7500 - val_loss: 0.9536
Epoch 33/40
12/12 - 1s - 45ms/step - accuracy: 0.9474 - loss: 0.1385 - val_accuracy: 0.7391 - val_loss: 0.8181
Epoch 34/40
12/12 - 1s - 42ms/step - accuracy: 0.9584 - loss: 0.1367 - val_accuracy: 0.7174 - val_loss: 0.8067
Epoch 35/40
12/12 - 1s - 43ms/step - accuracy: 0.9668 - loss: 0.1049 - val_accuracy: 0.7391 - val_loss: 0.8764
Epoch 36/40
12/12 - -0s - -3276us/step - accuracy: 0.9695 - loss: 0.1173 - val_accuracy: 0.6957 - val_loss: 1.0640
Epoch 37/40
12/12 - 0s - 38ms/step - accuracy: 0.9418 - loss: 0.1499 - val_accuracy: 0.7391 - val_loss: 1.0510
Epoch 38/40
12/12 - 1s - 49ms/step - accuracy: 0.9529 - loss: 0.1100 - val_accuracy: 0.7391 - val_loss: 0.9813
Epoch 39/

Fine-tuning the model for 15 epochs...
Epoch 1/15
12/12 - 3s - 213ms/step - accuracy: 0.9695 - loss: 0.1178 - val_accuracy: 0.7609 - val_loss: 0.9173
Epoch 2/15
12/12 - 6s - 493ms/step - accuracy: 0.9501 - loss: 0.1438 - val_accuracy: 0.7609 - val_loss: 0.9188
Epoch 3/15
12/12 - -5s - -445929us/step - accuracy: 0.9612 - loss: 0.1314 - val_accuracy: 0.7609 - val_loss: 0.9207
Epoch 4/15
12/12 - 1s - 48ms/step - accuracy: 0.9391 - loss: 0.1699 - val_accuracy: 0.7500 - val_loss: 0.9219
Epoch 5/15
12/12 - 0s - 37ms/step - accuracy: 0.9612 - loss: 0.1177 - val_accuracy: 0.7500 - val_loss: 0.9230
Epoch 6/15
12/12 - 1s - 44ms/step - accuracy: 0.9640 - loss: 0.1160 - val_accuracy: 0.7500 - val_loss: 0.9242
Epoch 7/15
12/12 - 1s - 45ms/step - accuracy: 0.9640 - loss: 0.1211 - val_accuracy: 0.7500 - val_loss: 0.9260
Epoch 8/15
12/12 - 0s - 41ms/step - accuracy: 0.9529 - loss: 0.1397 - val_accuracy: 0.7500 - val_loss: 0.9272
Epoch 9/15
12/12 - 1s - 45ms/step - accuracy: 0.9557 - loss: 0.1565 - val

In [22]:
# dont use this
model = tf.keras.models.load_model('best_model.h5')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpput8cgfr/assets


INFO:tensorflow:Assets written to: /tmp/tmpput8cgfr/assets


Saved artifact at '/tmp/tmpput8cgfr'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name='x_input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140303567871952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023194384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023190544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023186320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023193616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023192464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023184016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023189392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023191696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140302023183248: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1737508447.868421   94864 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1737508447.868467   94864 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-21 17:14:07.868602: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpput8cgfr
2025-01-21 17:14:07.869056: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-21 17:14:07.869062: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpput8cgfr
2025-01-21 17:14:07.872853: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-21 17:14:07.900200: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpput8cgfr
2025-01-21 17:14:07.907219: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 38619 microseconds.


In [27]:
# Load the trained model
model = tf.keras.models.load_model('best_model_bm_trainable_val_acc_95p.h5')

# Create the TFLiteConverter object
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable INT8 quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Specify representative dataset for proper quantization
# Define a representative dataset generator function
def representative_data_gen():
    for _ in range(100):  # Provide 100 representative samples
        # Use sample images from your dataset resized to the expected input shape
        # Replace with actual preprocessed images as numpy arrays
        yield [tf.random.uniform(shape=(1, 96, 96, 1), minval=0, maxval=1, dtype=tf.float32)]

converter.representative_dataset = representative_data_gen

# Ensure the model is fully quantized to INT8
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Specify that both input and output tensors should be INT8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model to TFLite format
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('model_int8.tflite', 'wb') as f:
    f.write(tflite_model)

print("INT8 quantized model saved as 'model_int8.tflite'.")

INFO:tensorflow:Assets written to: /tmp/tmpejanyv66/assets


INFO:tensorflow:Assets written to: /tmp/tmpejanyv66/assets


Saved artifact at '/tmp/tmpejanyv66'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name='x_input')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140301757982544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652362704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652355024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140301757984464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652359440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652355216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652350992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652357136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652351184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652362512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140306652360976: Tens

/mnt/c/Users/rtill/AI-venv/AIvenv/lib/python3.12/site-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1737512330.577733   94864 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1737512330.578062   94864 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-21 18:18:50.579033: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpejanyv66
2025-01-21 18:18:50.587175: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-21 18:18:50.587199: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpejanyv66
2025-01-21 18:18:50.676099: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-21 18:18:51.177025: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path

INT8 quantized model saved as 'model_int8.tflite'.


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
